# Mentoría DiploDatos 2026 - Clasificación de Clientes Masivos según su Comportamiento de Reclamos en Telecomunicaciones


### Introduccion

El proyecto consiste en desarrollar un modelo de clasificación de clientes para una empresa del sector de telecomunicaciones a partir de la información de sus reclamos, con el objetivo de identificar patrones en común sobre el servicio y definir estrategias de acción personalizadas para cada grupo, generando así valor comercial y fortaleciendo la toma de decisiones de negocio.

La iniciativa demuestra el valor de la clusterización y de los modelos no supervisados como herramientas fundamentales para comprender en profundidad el comportamiento y las necesidades de los usuarios. A partir de los segmentos generados, es posible planificar acciones concretas orientadas a mejorar la fidelización, impulsar ventas cruzadas y diseñar estrategias de retención más efectivas.

Es importante recordar que no todos los clientes son iguales. Aprovechar los datos de manera inteligente permite ofrecer experiencias más personalizadas, relevantes y satisfactorias, fortaleciendo la relación con cada uno de ellos.

### Inicializacion del entorno

In [1]:
!pip install pandas 


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install plotly


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install matplotlib


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install seaborn


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install scikit-learn


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
!pip install scipy


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
!pip install sqlalchemy


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
!pip install cx_Oracle


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
# Importación de librerías
import numpy as np
import pandas as pd
import itertools
import warnings
warnings.filterwarnings("ignore")
import io

# Para visualización
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
from plotly.figure_factory import create_table
init_notebook_mode(connected=True)

import matplotlib.pyplot as plt
import seaborn as sns

# Librerías de aprendizaje automático
from sklearn.cluster import KMeans, MeanShift, DBSCAN 
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import OneHotEncoder
import sklearn
from sklearn.impute import SimpleImputer
from scipy import sparse

# Librerías estadísticas y matemáticas
from scipy.stats import t
from scipy import stats
from math import pi

In [10]:
#Carga de datos
ruta = r"Dataset_reclamos.csv"

df = pd.read_csv(
    ruta,
    sep=";",                # separador de columnas
    quotechar='"',          # texto entre comillas
    engine="python",        # parser más tolerante
    encoding="utf-8-sig",   # prueba 1 de codificación
    on_bad_lines="skip",    # salta filas corruptas (pandas >= 1.3)
    dtype=str               # lee todo como texto (evita errores de tipos)
)

print(df.shape)

(1809697, 30)


In [11]:
#Listamos las variables para ver cuales son interesantes
print(len(df.columns), 'variables')
print(df.columns)

30 variables
Index(['TPF_ID', 'SERVICIO', 'TPF_PAIS', 'TPF_FECHA', 'TPF_PLAZA',
       'TPF_LOCALIDAD', 'TPF_MODELO_ONT', 'TPF_VEL_CONTRATADA',
       'TPF_SOFT_VERSION', 'TPF_FECHA_ACTIVA', 'TPF_BAND_STERING_WIFI',
       'TPF_ACTIVAR_WIFI', 'TPF_REPETIDOR_WIFI', 'TPF_CANT_REPE_WIFI',
       'TPF_POTENCIA_TX_GPON', 'TPF_POTENCIA_RX_GPON', 'TPF_OLT_GPON',
       'TPF_OLT_POTENCIA_TX', 'TPF_OLT_POTENCIA_RX', 'TPF_CORTE_ONT_LASTDW',
       'TPF_CORTE_ONT_UPTIME', 'TPF_ESTADO_GPON_CAT', 'TPF_CORTE_ONT_CAT',
       'TPF_CODIF_TIPO_CAT', 'TPF_TECNO_CAT5', 'TPF_SERV_CAT8', 'TPF_PROB_CAT',
       'TPF_CIERRE_CAT', 'DURACION_CORTE_EN_HORAS',
       'ANTIGUEDAD_CLIENTE_ANIOS'],
      dtype='object')


### Detalle del dataset por feature:
El dataset tiene información de reclamos de clientes masivos de servicios de telecomunicaciones, realizados durante 2025.

- tpf_id: identificación del reclamo (es único)
- servicio: servicio de telecomunicaciones comercializado masivamente (internet, IPTV, Telefonía,Apps-Streaming,Otros)
- tpf_pais: El servicio se comercializa en Paraguay y Argentina
- tpf_fecha: detalle de día,mes,año y hora del reclamo
- tpf_plaza: localidad donde está el nodo GPON del operador
- tpf_localidad: ciudad donde está el equipo del cliente
- tpf_modelo_ont: modelo de la ONT (Terminal Óptica de Red) que es el equipo que tiene el cliente en su casa
- tpf_vel_contratada: velocidad contratada por el cliente para el servicio de datos
- tpf_soft_version: versión del software de la ONT en los clientes que hicieron el reclamo
- tpf_fecha_activa: detalle de día,mes,año y hora del alta de la ONT
- tpf_band_stering_wifi: Indica si el cliente tiene activada la función Band Steering (la ONT mueve automáticamente al cliente entre las bandas 2.4 GHz y 5 GHz de acuerdo a congestión, capacidad de señal,etc.)
- tpf_activar_wifi: Indica si la red WiFi de la ONT está activada
- tpf_repetidor_wifi: Indica si el cliente tiene repetidores WiFi vinculados a su servicio
- tpf_cant_repe_wifi:si el cliente que reclamo tiene repetidor, indica cantidad
- tpf_potencia_tx_gpon: Es la potencia óptica que la ONT transmite hacia la OLT
- tpf_potencia_rx_gpon: Es la potencia óptica que la ONT recibe desde la OLT.Es el indicador más crítico de salud del enlace óptico
- tpf_olt_gpon: modelo de la OLT (Optical Line Terminal) que es el equipo ubicado en la central o nodo GPON del operador
- tpf_olt_potencia_tx: Es la potencia que la OLT transmite hacia la ONT
- tpf_olt_potencia_rx: Es la potencia que la OLT recibe desde la ONT
- tpf_corte_ont_lastdw: Es la fecha y hora en la que la ONT perdió señal (o dejó de responder); en ese momento se cayó.
- tpf_corte_ont_uptime: Es la última fecha y hora en la que la ONT fue vista en estado operativo previo al último evento de caída
- tpf_estado_gpon_cat: indica si el servicio GPON está Up o Down en el momento de registrarse el reclamo
- tpf_corte_ont_cat: causas del corte (energía, pérdida de sincronismo,etc)
- tpf_codif_tipo_cat: codificación del reclamo por tipo
- tpf_tecno_cat5:codificación del reclamo por tecnología (internet fijo, tv)
- tpf_serv_cat8: codificación del reclamo por servicio (telefonia,internet,STB)
- tpf_prob_cat: codificación del reclamo por tipo de problema (contenido,plataforma,pagos)
- tpf_cierre_cat: codificación de cierre (solucionado, AT)
- duracion_corte_en_horas: tiempo (en horas)desde que la ONT estuvo operativa por ultima vez y cuando se perdió señal
- antiguedad_cliente_anios: tiempo (en años) desde el alta del equipo y el reclamo


#### **Trabajo practico 2**: Analisis Exploratorio y Curación de datos
En esta etapa acondicionamos el dataset para alimentar el modelo:

##### Cuestiones a analizar y responder:
1. **Valores faltantes (Nan, NA o None)**: ¿Existen valores faltantes en el conjunto de datos? ¿Porqué piensan que ocurrió ésto?¿Deberíamos eliminar las filas/columnas o imputar valores?

2. **Outliers**: ¿Existen valores atipicos? ¿Que tecnica utilizarán para la deteccion y manejo de estos valores?

3. **Tratamiento de features**: Divida las variables en numericas y categoricas. ¿Es necesario codificar variables categoricas en numericas?

4. **Escalado y Normalización**: ¿Necesitamos escalar o normalizar las variables numéricas? ¿Qué técnicas de escalado o normalización deberíamos usar?

5. **Feature engineering**:¿Existe relacion entre las variables?¿Se elimina alguna variable? ¿Creen necesario reducir las variables a través de PCA?Justificar las ventajas y desventajas de aplicarlo

In [12]:
TOTAL = len(df) # denominador fijo que pediste

# Conteo de faltantes por columna
missing_count = df.isna().sum()

# Porcentaje de faltantes respecto a 73.850
missing_pct = (missing_count / TOTAL) * 100

# Tabla resumen ordenada
missing_summary = (
    pd.DataFrame({
        'faltantes': missing_count,
        'porcentaje_sobre_73850': missing_pct.round(2)
    })
    .sort_values('porcentaje_sobre_73850', ascending=False)
)

print("Valores faltantes y porcentaje (base 73.850):")
print(missing_summary)

Valores faltantes y porcentaje (base 73.850):
                          faltantes  porcentaje_sobre_73850
TPF_CANT_REPE_WIFI          1237043                   68.36
TPF_REPETIDOR_WIFI          1051892                   58.13
TPF_ACTIVAR_WIFI            1051892                   58.13
TPF_BAND_STERING_WIFI       1051892                   58.13
TPF_OLT_POTENCIA_RX          890026                   49.18
TPF_OLT_POTENCIA_TX          890065                   49.18
TPF_POTENCIA_TX_GPON         889453                   49.15
TPF_POTENCIA_RX_GPON         889453                   49.15
DURACION_CORTE_EN_HORAS      708399                   39.14
TPF_OLT_GPON                 464560                   25.67
TPF_SOFT_VERSION             398168                   22.00
TPF_CORTE_ONT_UPTIME         286953                   15.86
TPF_CORTE_ONT_LASTDW         286953                   15.86
TPF_ESTADO_GPON_CAT          265446                   14.67
TPF_PLAZA                     16797                   

1.Tratamiento Nan 
En base a los porcentajes de faltantes se debe decidir que hacer con ellos; este es un ejemplo de eliminación de variables, eliminación de filas e imputación (PUEDEN USTEDES PROBAR OTRAS ALTERNATIVAS)
- Eliminamos las 4 variables WiFi porque tiene mas de la mitad de faltantes
- Eliminamos filas con NaN en:
TPF_MODELO_ONT, TPF_VEL_CONTRATADA, TPF_FECHA_ACTIVA
- Reemplazamos NaN por "DESCONOCIDO" en variables de texto:
TPF_PLAZA
TPF_LOCALIDAD
TPF_SOFT_VERSION
TPF_ESTADO_GPON_CAT
TPF_OLT_GPON
TPF_CORTE_ONT_LASTDW
TPF_CORTE_ONT_UPTIME
- ReemplazaMOS NaN con 0 en las Variables NUMÉRICAS (potencias ONT/OLT, DURACION_CORTE_EN_HORAS,ANTIGUEDAD_CLIENTE_AÑOS)

In [ ]:
# Hacemos el tratamiento planteado
# 1) Eliminar columnas WiFi
cols_wifi = [
    'TPF_BAND_STERING_WIFI',
    'TPF_ACTIVAR_WIFI',
    'TPF_REPETIDOR_WIFI',
    'TPF_CANT_REPE_WIFI'
]

df = df.drop(columns=[c for c in cols_wifi if c in df.columns], errors='ignore')

# 2) Eliminar filas con NaN en columnas obligatorias
cols_obligatorias = [
    'TPF_MODELO_ONT',
    'TPF_VEL_CONTRATADA',
    'TPF_FECHA_ACTIVA'
]

df = df.dropna(subset=[c for c in cols_obligatorias if c in df.columns])

# 3) Reemplazar NaN en variables de texto con "DESCONOCIDO"
cols_texto = [
    'TPF_PLAZA',
    'TPF_LOCALIDAD',
    'TPF_SOFT_VERSION',
    'TPF_ESTADO_GPON_CAT',
    'TPF_OLT_GPON',
    'TPF_CORTE_ONT_LASTDW',
    'TPF_CORTE_ONT_UPTIME',
]

for c in cols_texto:
    if c in df.columns:
        df[c] = df[c].fillna('DESCONOCIDO')

# 4) Reemplazar NaN en variables numéricas con 0
cols_numericas = [
    'TPF_POTENCIA_TX_GPON',
    'TPF_POTENCIA_RX_GPON',
    'TPF_OLT_POTENCIA_TX',
    'TPF_OLT_POTENCIA_RX',
    'DURACION_CORTE_EN_HORAS',
    'ANTIGUEDAD_CLIENTE_ANIOS'
]

for c in cols_numericas:
    if c in df.columns:
        # Normalizar si vienen con coma decimal
        df[c] = (df[c].astype(str)
                        .str.replace(',', '.', regex=False))
        df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)

df

,TPF_ID,SERVICIO,TPF_PAIS,TPF_FECHA,TPF_PLAZA,TPF_LOCALIDAD,TPF_MODELO_ONT,TPF_VEL_CONTRATADA,TPF_SOFT_VERSION,TPF_FECHA_ACTIVA,...,TPF_CORTE_ONT_UPTIME,TPF_ESTADO_GPON_CAT,TPF_CORTE_ONT_CAT,TPF_CODIF_TIPO_CAT,TPF_TECNO_CAT5,TPF_SERV_CAT8,TPF_PROB_CAT,TPF_CIERRE_CAT,DURACION_CORTE_EN_HORAS,ANTIGUEDAD_CLIENTE_ANIOS
0,15157760,Internet,Argentina,30/12/2025 23:57:31,SAN MIGUEL DE TUCUMAN,SAN MIGUEL DE TUCUMAN,HG8245W5-6T,100 Mb,V5R020C00S448,06/10/2022 20:13:54,...,26/12/2025 18:48:18,Down,Pérdida de sincronismo / señal,Cliente,Internet Fijo,Telefonía / Móvil,Contenidos / Cortes / Intermitencia,AT / Visita / Coordinación,99.91,3.23
1,15157759,Otros,Argentina,30/12/2025 23:57:27,ROSARIO,ROSARIO,ZTE ZXV10-B866V2,100 Mb,DESCONOCIDO,11/07/2022 12:33:38,...,24/12/2025 00:59:11,Up,Corte energético,Cliente,TV/OTT,STB / Deco,Plataforma / Apps / Streaming,Solucionado,0.00,3.47
2,15157758,Internet,Paraguay,30/12/2025 23:56:22,LUQUE,LUQUE,HG8145X6-10,100 Mb,V5R022C10S342,22/04/2024 13:49:51,...,30/12/2025 23:37:32,Up,Corte energético,Cliente,Internet Fijo,Internet Fijo / Acceso,Contenidos / Cortes / Intermitencia,AT / Visita / Coordinación,0.03,1.69
3,15157757,Internet,Argentina,30/12/2025 23:55:34,VICENTE LOPEZ,FLORIDA,F6600PV9.0,100 Mb,V5R020C00S422,14/06/2018 12:10:19,...,"Momentaneamente, no se reporta este valor",Up,Señalización / Protocolos,Cliente,Internet Fijo,Cliente / Administrativo,Pagos / Cobranza,Escalamiento / Derivación,0.00,7.55
4,15157756,Internet,Argentina,30/12/2025 23:55:32,SANTA FE,SANTA FE,HG8245W5,100 Mb,V5R020C00S422,12/02/2020 11:08:58,...,30/12/2025 03:21:53,Down,Pérdida de sincronismo / señal,Cliente,Internet Fijo,Telefonía / Móvil,Contenidos / Cortes / Intermitencia,AT / Visita / Coordinación,10.14,5.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1809692,12194442,Internet,Argentina,1/1/2025 07:13:51,JUJUY,SAN SALVADOR DE JUJUY,HG8145X6-10,300 Mb,V5R022C00S127,28/09/2024 16:40:10,...,"Momentaneamente, no se reporta este valor",Down,Señalización / Protocolos,Cliente,Internet Fijo,Telefonía / Móvil,Contenidos / Cortes / Intermitencia,AT / Visita / Coordinación,0.00,0.26
1809693,12194441,Internet,Paraguay,1/1/2025 07:11:09,SAN LORENZO,SAN LORENZO,HG8245W5-6T,100 Mb,V5R020C00S448,31/10/2022 19:32:38,...,01/01/2025 07:08:18,Up,Corte energético,Cliente,Internet Fijo,Internet Fijo / Acceso,Acceso / Internet / GPON-ONT,Reset / Reinicio,0.01,2.17
1809694,12194440,Internet,Argentina,1/1/2025 06:52:59,VICENTE LOPEZ,FLORIDA OESTE,HG8245W5-6T,50 Mb,V5R020C00S448,02/07/2021 18:16:49,...,30/12/2024 10:13:37,Down,Pérdida de sincronismo / señal,Cliente,Internet Fijo,Telefonía / Móvil,Contenidos / Cortes / Intermitencia,AT / Visita / Coordinación,42.03,3.50
1809695,12194439,Internet,Paraguay,1/1/2025 06:45:52,MARIANO ROQUE ALONSO,MARIANO ROQUE ALONSO,HG8145X6-10,100 Mb,V5R022C00S127,13/08/2024 11:45:03,...,31/12/2024 20:28:04,Up,Reinicio / Reset de ONT,Cliente,Internet Fijo,Cliente / Administrativo,Administrativo / Comercial / Consultas,Administrativo / Comercial,0.20,0.39


In [ ]:
# Verificamos que no hay faltantes en el df resultante
missing_values = df.isna().sum()
print("Valores Faltantes por Columna:")
print(missing_values)

Valores Faltantes por Columna:
TPF_ID                      0
SERVICIO                    0
TPF_PAIS                    0
TPF_FECHA                   0
TPF_PLAZA                   0
TPF_LOCALIDAD               0
TPF_MODELO_ONT              0
TPF_VEL_CONTRATADA          0
TPF_SOFT_VERSION            0
TPF_FECHA_ACTIVA            0
TPF_POTENCIA_TX_GPON        0
TPF_POTENCIA_RX_GPON        0
TPF_OLT_GPON                0
TPF_OLT_POTENCIA_TX         0
TPF_OLT_POTENCIA_RX         0
TPF_CORTE_ONT_LASTDW        0
TPF_CORTE_ONT_UPTIME        0
TPF_ESTADO_GPON_CAT         0
TPF_CORTE_ONT_CAT           0
TPF_CODIF_TIPO_CAT          0
TPF_TECNO_CAT5              0
TPF_SERV_CAT8               0
TPF_PROB_CAT                0
TPF_CIERRE_CAT              0
DURACION_CORTE_EN_HORAS     0
ANTIGUEDAD_CLIENTE_ANIOS    0
dtype: int64


In [ ]:
#Armamos el df con variables numéricas y categoricas por separado (decidir cuales vamos a usar)-aislamos fechas y ID
numeric_df = df[
    [
        'TPF_POTENCIA_TX_GPON',
        'TPF_POTENCIA_RX_GPON',
        'TPF_OLT_POTENCIA_TX',
        'TPF_OLT_POTENCIA_RX',
        'DURACION_CORTE_EN_HORAS',
        'ANTIGUEDAD_CLIENTE_ANIOS'
    ]
].copy()


cat_df = df[
    [
        'SERVICIO',
        'TPF_PAIS',
        'TPF_PLAZA',
        'TPF_LOCALIDAD',
        'TPF_MODELO_ONT',
        'TPF_VEL_CONTRATADA',
        'TPF_SOFT_VERSION',
        'TPF_OLT_GPON',
        'TPF_ESTADO_GPON_CAT',
        'TPF_CORTE_ONT_CAT',
        'TPF_CODIF_TIPO_CAT',
        'TPF_TECNO_CAT5',
        'TPF_SERV_CAT8',
        'TPF_PROB_CAT',
        'TPF_CIERRE_CAT'
    ]
].copy()



In [ ]:
#Codificación de variables categoricas a numericas -  One-Hot *sparse* con límite de cardinalidad 
imputer_cat = SimpleImputer(strategy='most_frequent')
cat_imputed = imputer_cat.fit_transform(cat_df)  # array de objetos pero compacto
ohe = OneHotEncoder(
    handle_unknown='infrequent_if_exist',
    max_categories=100,        # o tope duro por columna (ajustá a tu gusto)
    sparse_output=True,        # MATRIZ DISPERSA (NO DENSIFICAR)
    dtype=np.uint8             # 0/1 ocupa muchísimo menos que float64
)
X_cat_sparse = ohe.fit_transform(cat_imputed)   

In [ ]:
# Armamos el nuevo df codificado
cat_cols = list(cat_df.columns)
cat_feature_names = ohe.get_feature_names_out(cat_cols)
df_cat_ohe = pd.DataFrame.sparse.from_spmatrix(
    X_cat_sparse,
    index=cat_df.index,           
    columns=cat_feature_names
)
print(df_cat_ohe.shape)    

(1807748, 399)
SERVICIO_Apps - Streaming    Sparse[uint8, 0]
SERVICIO_IPTV                Sparse[uint8, 0]
SERVICIO_Internet            Sparse[uint8, 0]
SERVICIO_Otros               Sparse[uint8, 0]
SERVICIO_Telefonía           Sparse[uint8, 0]
dtype: object


In [ ]:
# Escalamos las variables numéricas
scaler = StandardScaler()
numeric_scaled = scaler.fit_transform(numeric_df)
df_numeric_scaled = pd.DataFrame(numeric_scaled, columns=numeric_df.columns, index=df.index)

# Armar el DataFrame final: numéricas escaladas + dummies
X_final = pd.concat([df_numeric_scaled, df_cat_ohe], axis=1)

X_final.head()

,TPF_POTENCIA_TX_GPON,TPF_POTENCIA_RX_GPON,TPF_OLT_POTENCIA_TX,TPF_OLT_POTENCIA_RX,DURACION_CORTE_EN_HORAS,ANTIGUEDAD_CLIENTE_ANIOS,SERVICIO_Apps - Streaming,SERVICIO_IPTV,SERVICIO_Internet,SERVICIO_Otros,...,TPF_CIERRE_CAT_Ausente / No en domicilio,TPF_CIERRE_CAT_Cobertura / Energía / Clima,TPF_CIERRE_CAT_Comunicaciones / Contacto,TPF_CIERRE_CAT_Configuración WiFi/Canal/Dispositivo,TPF_CIERRE_CAT_Escalamiento / Derivación,TPF_CIERRE_CAT_Otros,TPF_CIERRE_CAT_Reposición / Instalación / Upgrade,TPF_CIERRE_CAT_Reset / Reinicio,TPF_CIERRE_CAT_Servicios/Perfiles (DMZ/BRIDGE/IP/Gamer/Packs),TPF_CIERRE_CAT_Solucionado
0,-0.004354,0.984985,-1.011370,0.095597,0.420858,0.338303,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.004354,0.984985,-1.011370,0.095597,-0.197465,0.449849,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
2,0.002252,-2.023848,1.059661,-0.181028,-0.197279,-0.377451,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,-0.004354,0.984985,-1.011370,0.095597,-0.197465,2.346131,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
4,-0.004354,0.984985,-1.011370,0.095597,-0.134710,1.569957,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# Si quiero grabar el dataset para llevar al TP3 
#X_final.to_csv("X_final.csv", index=False)

In [ ]:
#Calculamos varianza para ver con cuantas variables podemos reducir el dataset
pca = PCA()
pca.fit(X_final)
print("Varianza explicada por cada componente (estandarizado):", pca.explained_variance_ratio_.cumsum())

Varianza explicada por cada componente (estandarizado): [0.17164684 0.25707631 0.32631666 0.38942887 0.45162095 0.50834182
 0.54229422 0.57340861 0.5971988  0.61895606 0.63632676 0.65292063
 0.66890353 0.68373422 0.69723849 0.71051891 0.72178428 0.7324119
 0.74227481 0.75132981 0.76021774 0.76898614 0.77736155 0.78509062
 0.79232391 0.79953229 0.80579912 0.81171343 0.8175036  0.8227332
 0.8277179  0.83262188 0.83714665 0.84116683 0.84507382 0.84887741
 0.85250235 0.85602233 0.85949622 0.86275059 0.86579635 0.8687899
 0.87170588 0.87454226 0.87725384 0.87992069 0.88248354 0.88502631
 0.88743275 0.88976776 0.89201581 0.89416122 0.89625092 0.89830023
 0.90031236 0.90230341 0.90426474 0.90620402 0.908068   0.90989111
 0.91170301 0.91349704 0.91522239 0.91692968 0.91858599 0.92019669
 0.92178927 0.92335798 0.92479957 0.9262213  0.92761857 0.92898642
 0.93034258 0.93166936 0.93292129 0.93414867 0.93534418 0.93641581
 0.93747775 0.938495   0.93940583 0.94029367 0.9411528  0.94200166
 0.942842

In [ ]:
#Aplicamos reducción de variables (de acuerdo a la varianza cambiar el numero de componentes)
pca=PCA(n_components=15)
# Transformar los datos y obtener las componentes principales
projected_features_std = pca.fit_transform(X_final)

# Crear un DataFrame con las dos primeras componentes principales
df_escalado_pca = pd.DataFrame(projected_features_std)
df_escalado_pca

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,2.103081,1.095340,0.151255,-0.019230,-0.083130,-0.172021,-0.852670,0.850287,0.532713,-0.297765,-0.271704,-0.200309,0.101404,-0.139375,-0.000098
1,0.666081,-1.856620,1.496687,-0.101215,-0.443268,0.646111,-0.351870,-0.226594,0.701541,-0.311182,-0.259105,0.714482,-0.928677,-0.044289,-0.462862
2,-2.002942,0.875542,-1.302245,0.036542,0.192750,-0.125569,-0.601982,-0.739481,0.893802,-0.109713,0.562209,-0.016844,-0.124285,0.243558,0.882929
3,1.094006,0.684454,2.105428,0.037149,0.161122,-0.790363,0.971995,-0.927508,0.663216,-0.747371,0.413658,0.400342,0.061028,-0.379267,-0.678698
4,1.931909,1.419586,1.092253,0.003740,0.013473,-0.567607,-1.165371,-0.827147,0.536697,0.039028,0.425920,-0.342484,0.067492,0.077345,-0.564375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1807743,1.837030,-0.060013,-1.530878,0.013634,0.113136,-0.599095,-0.978513,-0.274536,-0.538241,0.327543,0.319163,-0.070460,-0.301612,-0.169662,-0.235224
1807744,-1.645925,0.798906,-0.448341,0.083978,0.369102,-0.294406,0.270215,0.924490,1.297381,0.100930,0.546604,-0.266690,-0.497138,0.793619,0.330674
1807745,2.044962,1.066587,0.393857,-0.002832,-0.006005,-0.488130,-0.815237,1.007671,-0.476010,0.247635,0.081846,-0.064105,0.242740,-0.195953,0.397072
1807746,-1.213176,-0.147182,-1.288528,0.049113,0.233327,-0.088330,0.133749,-0.379573,0.798032,-0.961231,1.080106,-0.624612,0.581285,-0.638419,0.023776


### Conclusiones y Observaciones